In [12]:
import pandas as pd
import numpy as np

In [13]:
owid_source = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'

In [14]:
data = pd.read_csv(owid_source)

In [15]:
data

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
0,AFG,Asia,Afghanistan,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
1,AFG,Asia,Afghanistan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
2,AFG,Asia,Afghanistan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
3,AFG,Asia,Afghanistan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
4,AFG,Asia,Afghanistan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30216,NaN,NaN,International,2020-02-28,705.0,0.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30217,NaN,NaN,International,2020-02-29,705.0,0.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30218,NaN,NaN,International,2020-03-01,705.0,0.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30219,NaN,NaN,International,2020-03-02,705.0,0.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
LocationKeys = 	['iso_code','location']
DailyKeys = ['date','total_cases','new_cases', 'total_deaths', 'new_deaths']
DailyKeysExtra = ['total_cases_per_million','new_cases_per_million','total_deaths_per_million','new_deaths_per_million']
PermanentKeys = ['population','population_density', 'gdp_per_capita']
PermanentKeysExtra = ['median_age','aged_65_older','aged_70_older','extreme_poverty','cvd_death_rate','diabetes_prevalence','female_smokers','male_smokers','handwashing_facilities','hospital_beds_per_thousand']

In [17]:
df = data

In [18]:
df = df.drop(df[df['location']=='International'].index)
df = df.drop(df[df['location']=='World'].index)

In [19]:
case_threshold = 50

In [20]:
df = df.drop(df[df['total_cases']<50].index)

In [21]:
data_avail = df.groupby(['location']).count()

In [22]:
data_avail = data_avail//7
data_avail = data_avail.reset_index()

In [23]:
data_avail = data_avail.drop(data_avail[data_avail['date']<10].index)

In [24]:
data_avail = data_avail.reset_index(drop=True)
data_avail['location']

0      Afghanistan
1          Albania
2          Algeria
3          Andorra
4        Argentina
          ...     
163        Uruguay
164     Uzbekistan
165      Venezuela
166        Vietnam
167         Zambia
Name: location, Length: 168, dtype: object

In [25]:
df = df.drop(df[~df['location'].isin(data_avail['location'])].index)

In [26]:
df["date"] = pd.to_datetime(df["date"])

In [27]:
df

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
76,AFG,Asia,Afghanistan,2020-03-26,75.0,33.0,1.0,0.0,1.927,0.848,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.26,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
77,AFG,Asia,Afghanistan,2020-03-27,75.0,0.0,1.0,0.0,1.927,0.000,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.26,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
78,AFG,Asia,Afghanistan,2020-03-28,91.0,16.0,2.0,1.0,2.338,0.411,0.051,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.59,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
79,AFG,Asia,Afghanistan,2020-03-29,106.0,15.0,3.0,1.0,2.723,0.385,0.077,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.59,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
80,AFG,Asia,Afghanistan,2020-03-30,114.0,8.0,4.0,1.0,2.928,0.206,0.103,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.59,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29839,ZMB,Africa,Zambia,2020-07-10,1895.0,0.0,42.0,0.0,103.079,0.000,2.285,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.93,18383956.0,22.995,17.7,2.480,1.542,3689.251,57.5,234.499,3.94,3.1,24.7,13.938,2.0,63.89
29840,ZMB,Africa,Zambia,2020-07-11,1895.0,0.0,42.0,0.0,103.079,0.000,2.285,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18383956.0,22.995,17.7,2.480,1.542,3689.251,57.5,234.499,3.94,3.1,24.7,13.938,2.0,63.89
29841,ZMB,Africa,Zambia,2020-07-12,1895.0,0.0,42.0,0.0,103.079,0.000,2.285,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18383956.0,22.995,17.7,2.480,1.542,3689.251,57.5,234.499,3.94,3.1,24.7,13.938,2.0,63.89
29842,ZMB,Africa,Zambia,2020-07-13,1895.0,0.0,42.0,0.0,103.079,0.000,2.285,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18383956.0,22.995,17.7,2.480,1.542,3689.251,57.5,234.499,3.94,3.1,24.7,13.938,2.0,63.89


In [28]:
df.groupby('location')['date'].groups.keys()

dict_keys(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bermuda', 'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Congo', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Estonia', 'Ethiopia', 'Faeroe Islands', 'Finland', 'France', 'French Polynesia', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'J

In [29]:
def tranf(obj):
  date_ret = obj.index
  return(date_ret - date_ret[0])

def tranf2(obj):
  date_ret = obj.index
  return((date_ret - date_ret[0])//7)

In [30]:
df['days'] = np.zeros(len(df), dtype='int64')
df['days'] = df.groupby('location')['date'].transform(tranf)
df['week'] = np.zeros(len(df), dtype='int64')
df['week'] = df.groupby('location')['date'].transform(tranf2)

In [31]:
Keys=LocationKeys+['week']+DailyKeys+DailyKeysExtra
train_data = df[Keys].reset_index(drop=True)

In [32]:
Keys=LocationKeys+['days']+DailyKeys+DailyKeysExtra
train_data_days = df[Keys].reset_index(drop=True)

In [33]:
df_melted = train_data.groupby(['location','week']).sum().reset_index()

In [34]:
df_melted_days = train_data_days.groupby(['location','days']).sum().reset_index()

In [35]:
df_melted

,location,week,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million
0,Afghanistan,0,768.0,124.0,19.0,3.0,19.729,3.186,0.489,0.078
1,Afghanistan,1,1935.0,201.0,42.0,7.0,49.708,5.164,1.080,0.180
2,Afghanistan,2,3969.0,347.0,124.0,12.0,101.957,8.914,3.184,0.308
3,Afghanistan,3,6450.0,378.0,218.0,13.0,165.690,9.710,5.601,0.334
4,Afghanistan,4,10332.0,735.0,346.0,24.0,265.411,18.880,8.888,0.617
...,...,...,...,...,...,...,...,...,...,...
2858,Zambia,8,9651.0,164.0,75.0,1.0,524.968,8.920,4.078,0.054
2859,Zambia,9,10183.0,81.0,98.0,7.0,553.907,4.407,5.329,0.381
2860,Zambia,10,11045.0,135.0,170.0,12.0,600.795,7.342,9.247,0.652
2861,Zambia,11,12213.0,263.0,246.0,12.0,664.329,14.306,13.383,0.653


In [36]:
# df_melted = df_melted[df_melted['week']<=10]

In [37]:
df_unmelted = df_melted.pivot(index='location', columns='week')

In [38]:
df_unmelted.to_csv('weekwise_exp.csv')

In [ ]:
col_names = list(df_unmelted.columns)


In [ ]:
col_names = [(col[0],str(col[1])) for col in col_names]

In [ ]:
col_names = ['_'.join(col).strip() for col in col_names]

In [ ]:
df_unmelted.columns = col_names

In [ ]:
df_unmelted.reset_index()

,location,total_cases_0,total_cases_1,total_cases_2,total_cases_3,total_cases_4,total_cases_5,total_cases_6,total_cases_7,total_cases_8,total_cases_9,total_cases_10,total_cases_11,total_cases_12,total_cases_13,total_cases_14,total_cases_15,total_cases_16,total_cases_17,total_cases_18,total_cases_19,total_cases_20,total_cases_21,total_cases_22,total_cases_23,new_cases_0,new_cases_1,new_cases_2,new_cases_3,new_cases_4,new_cases_5,new_cases_6,new_cases_7,new_cases_8,new_cases_9,new_cases_10,new_cases_11,new_cases_12,new_cases_13,new_cases_14,...,total_deaths_per_million_8,total_deaths_per_million_9,total_deaths_per_million_10,total_deaths_per_million_11,total_deaths_per_million_12,total_deaths_per_million_13,total_deaths_per_million_14,total_deaths_per_million_15,total_deaths_per_million_16,total_deaths_per_million_17,total_deaths_per_million_18,total_deaths_per_million_19,total_deaths_per_million_20,total_deaths_per_million_21,total_deaths_per_million_22,total_deaths_per_million_23,new_deaths_per_million_0,new_deaths_per_million_1,new_deaths_per_million_2,new_deaths_per_million_3,new_deaths_per_million_4,new_deaths_per_million_5,new_deaths_per_million_6,new_deaths_per_million_7,new_deaths_per_million_8,new_deaths_per_million_9,new_deaths_per_million_10,new_deaths_per_million_11,new_deaths_per_million_12,new_deaths_per_million_13,new_deaths_per_million_14,new_deaths_per_million_15,new_deaths_per_million_16,new_deaths_per_million_17,new_deaths_per_million_18,new_deaths_per_million_19,new_deaths_per_million_20,new_deaths_per_million_21,new_deaths_per_million_22,new_deaths_per_million_23
0,Afghanistan,768.0,1935.0,3969.0,6450.0,10332.0,17746.0,28822.0,44758.0,69621.0,101140.0,136559.0,117447.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.0,201.0,347.0,378.0,735.0,1397.0,1743.0,2686.0,4178.0,4678.0,4950.0,3307.0,NaN,NaN,NaN,...,37.454,44.928,60.110,56.488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.078,0.180,0.308,0.334,0.617,0.899,0.820,1.310,1.079,1.286,2.928,2.234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,470.0,1138.0,2000.0,2864.0,3603.0,4606.0,5391.0,5871.0,6321.0,6798.0,7517.0,8378.0,9689.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,123.0,149.0,85.0,116.0,164.0,69.0,73.0,78.0,52.0,139.0,109.0,275.0,NaN,NaN,...,75.404,75.752,79.922,80.965,85.137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.347,2.778,3.821,0.694,1.041,0.694,1.042,0.000,0.000,0.347,0.347,0.347,0.694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,722.0,2537.0,6875.0,11629.0,16063.0,21211.0,27943.0,36315.0,45122.0,54185.0,62849.0,68735.0,74147.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.0,310.0,809.0,594.0,715.0,753.0,1092.0,1249.0,1296.0,1287.0,1088.0,760.0,765.0,NaN,NaN,...,84.354,91.970,100.704,108.802,118.354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.297,0.320,2.759,3.215,1.870,1.139,0.867,0.890,1.050,1.187,1.208,1.230,1.367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Andorra,701.0,2067.0,3295.0,4313.0,4902.0,5137.0,5230.0,5278.0,5326.0,5337.0,5427.0,5963.0,4263.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,212.0,169.0,114.0,58.0,26.0,8.0,7.0,3.0,2.0,81.0,8.0,1.0,NaN,NaN,...,4542.807,4620.462,4620.462,4620.462,3300.330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,155.309,129.422,116.481,77.654,51.769,64.710,25.884,38.827,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Argentina,808.0,3480.0,7734.0,12698.0,17385.0,22971.0,29745.0,36565.0,47592.0,66110.0,98109.0,134087.0,183516.0,31564.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,520.0,706.0,686.0,691.0,939.0,901.0,1095.0,2029.0,3548.0,4861.0,5806.0,8275.0,1282.0,NaN,...,51.442,62.794,76.534,91.049,114.480,18.431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.044,0.332,0.531,1.018,0.952,1.172,1.151,1.394,1.394,1.814,1.837,2.656,3.696,0.398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
df_merge = df[['location']+PermanentKeys+PermanentKeysExtra]

In [ ]:
df_merge = df_merge.drop_duplicates() 

In [ ]:
df_merge

,location,population,population_density,gdp_per_capita,median_age,aged_65_older,aged_70_older,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
76,Afghanistan,38928341.0,54.422,1803.987,18.6,2.581,1.337,NaN,597.029,9.59,NaN,NaN,37.746,0.50
166,Albania,2877800.0,104.871,11803.431,38.0,13.188,8.643,1.1,304.195,10.08,7.1,51.2,NaN,2.89
329,Algeria,43851043.0,17.348,13913.839,29.1,6.211,3.857,0.5,278.364,6.73,0.7,30.4,83.741,1.90
425,Andorra,77265.0,163.755,NaN,NaN,NaN,NaN,NaN,109.135,7.97,29.0,37.8,NaN,NaN
778,Argentina,45195777.0,16.177,18933.907,31.9,11.198,7.441,0.6,191.032,5.50,16.2,27.7,NaN,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22763,United States,331002647.0,35.608,54225.446,38.3,15.413,9.732,1.2,151.089,10.79,19.1,24.6,NaN,2.77
22961,Uruguay,3473727.0,19.751,20551.409,35.6,14.655,10.361,0.1,160.708,6.93,14.0,19.9,NaN,2.80
23060,Uzbekistan,33469199.0,76.134,6253.104,28.2,4.469,2.873,NaN,724.417,7.57,1.3,24.7,NaN,4.00
23245,Venezuela,28435943.0,36.253,16745.022,29.0,6.614,3.915,NaN,204.850,6.47,NaN,NaN,NaN,0.80


In [ ]:
result = pd.merge(df_unmelted, df_merge, how='left', on=['location'])

In [ ]:
result[result.isnull().any(axis=1)]

,location,total_cases_0,total_cases_1,total_cases_2,total_cases_3,total_cases_4,total_cases_5,total_cases_6,total_cases_7,total_cases_8,total_cases_9,total_cases_10,total_cases_11,total_cases_12,total_cases_13,total_cases_14,total_cases_15,total_cases_16,total_cases_17,total_cases_18,total_cases_19,total_cases_20,total_cases_21,total_cases_22,total_cases_23,new_cases_0,new_cases_1,new_cases_2,new_cases_3,new_cases_4,new_cases_5,new_cases_6,new_cases_7,new_cases_8,new_cases_9,new_cases_10,new_cases_11,new_cases_12,new_cases_13,new_cases_14,...,total_deaths_per_million_21,total_deaths_per_million_22,total_deaths_per_million_23,new_deaths_per_million_0,new_deaths_per_million_1,new_deaths_per_million_2,new_deaths_per_million_3,new_deaths_per_million_4,new_deaths_per_million_5,new_deaths_per_million_6,new_deaths_per_million_7,new_deaths_per_million_8,new_deaths_per_million_9,new_deaths_per_million_10,new_deaths_per_million_11,new_deaths_per_million_12,new_deaths_per_million_13,new_deaths_per_million_14,new_deaths_per_million_15,new_deaths_per_million_16,new_deaths_per_million_17,new_deaths_per_million_18,new_deaths_per_million_19,new_deaths_per_million_20,new_deaths_per_million_21,new_deaths_per_million_22,new_deaths_per_million_23,population,population_density,gdp_per_capita,median_age,aged_65_older,aged_70_older,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
0,Afghanistan,768.0,1935.0,3969.0,6450.0,10332.0,17746.0,28822.0,44758.0,69621.0,101140.0,136559.0,117447.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.0,201.0,347.0,378.0,735.0,1397.0,1743.0,2686.0,4178.0,4678.0,4950.0,3307.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.078,0.180,0.308,0.334,0.617,0.899,0.820,1.310,1.079,1.286,2.928,2.234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,1803.987,18.6,2.581,1.337,NaN,597.029,9.59,NaN,NaN,37.746,0.50
1,Albania,470.0,1138.0,2000.0,2864.0,3603.0,4606.0,5391.0,5871.0,6321.0,6798.0,7517.0,8378.0,9689.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,123.0,149.0,85.0,116.0,164.0,69.0,73.0,78.0,52.0,139.0,109.0,275.0,NaN,NaN,...,NaN,NaN,NaN,0.347,2.778,3.821,0.694,1.041,0.694,1.042,0.000,0.000,0.347,0.347,0.347,0.694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2877800.0,104.871,11803.431,38.0,13.188,8.643,1.1,304.195,10.08,7.1,51.2,NaN,2.89
2,Algeria,722.0,2537.0,6875.0,11629.0,16063.0,21211.0,27943.0,36315.0,45122.0,54185.0,62849.0,68735.0,74147.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.0,310.0,809.0,594.0,715.0,753.0,1092.0,1249.0,1296.0,1287.0,1088.0,760.0,765.0,NaN,NaN,...,NaN,NaN,NaN,0.297,0.320,2.759,3.215,1.870,1.139,0.867,0.890,1.050,1.187,1.208,1.230,1.367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43851043.0,17.348,13913.839,29.1,6.211,3.857,0.5,278.364,6.73,0.7,30.4,83.741,1.90
3,Andorra,701.0,2067.0,3295.0,4313.0,4902.0,5137.0,5230.0,5278.0,5326.0,5337.0,5427.0,5963.0,4263.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,212.0,169.0,114.0,58.0,26.0,8.0,7.0,3.0,2.0,81.0,8.0,1.0,NaN,NaN,...,NaN,NaN,NaN,0.000,155.309,129.422,116.481,77.654,51.769,64.710,25.884,38.827,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77265.0,163.755,NaN,NaN,NaN,NaN,NaN,109.135,7.97,29.0,37.8,NaN,NaN
4,Argentina,808.0,3480.0,7734.0,12698.0,17385.0,22971.0,29745.0,36565.0,47592.0,66110.0,98109.0,134087.0,183516.0,31564.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,520.0,706.0,686.0,691.0,939.0,901.0,1095.0,2029.0,3548.0,4861.0,5806.0,8275.0,1282.0,NaN,...,NaN,NaN,NaN,0.044,0.332,0.531,1.018,0.952,1.172,1.151,1.394,1.394,1.814,1.837,2.656,3.696,0.398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45195777.0,16.177,18933.907,31.9,11.198,7.441,0.6,191.032,5.50,16.2,27.7,NaN,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
result.to_csv('data_extra_features.csv')

In [ ]:
df_melted_days

,location,days,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million
0,Afghanistan,0,75.0,33.0,1.0,0.0,1.927,0.848,0.026,0.000
1,Afghanistan,1,75.0,0.0,1.0,0.0,1.927,0.000,0.026,0.000
2,Afghanistan,2,91.0,16.0,2.0,1.0,2.338,0.411,0.051,0.026
3,Afghanistan,3,106.0,15.0,3.0,1.0,2.723,0.385,0.077,0.026
4,Afghanistan,4,114.0,8.0,4.0,1.0,2.928,0.206,0.103,0.026
...,...,...,...,...,...,...,...,...,...,...
12450,Vietnam,100,332.0,0.0,0.0,0.0,3.411,0.000,0.000,0.000
12451,Vietnam,101,332.0,0.0,0.0,0.0,3.411,0.000,0.000,0.000
12452,Vietnam,102,333.0,1.0,0.0,0.0,3.421,0.010,0.000,0.000
12453,Vietnam,103,334.0,1.0,0.0,0.0,3.431,0.010,0.000,0.000


In [ ]:
df_melted_days

,location,days,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million
0,Afghanistan,0,75.0,33.0,1.0,0.0,1.927,0.848,0.026,0.000
1,Afghanistan,1,75.0,0.0,1.0,0.0,1.927,0.000,0.026,0.000
2,Afghanistan,2,91.0,16.0,2.0,1.0,2.338,0.411,0.051,0.026
3,Afghanistan,3,106.0,15.0,3.0,1.0,2.723,0.385,0.077,0.026
4,Afghanistan,4,114.0,8.0,4.0,1.0,2.928,0.206,0.103,0.026
...,...,...,...,...,...,...,...,...,...,...
12450,Vietnam,100,332.0,0.0,0.0,0.0,3.411,0.000,0.000,0.000
12451,Vietnam,101,332.0,0.0,0.0,0.0,3.411,0.000,0.000,0.000
12452,Vietnam,102,333.0,1.0,0.0,0.0,3.421,0.010,0.000,0.000
12453,Vietnam,103,334.0,1.0,0.0,0.0,3.431,0.010,0.000,0.000


In [ ]:
df_melted_days['new_cases_ma'] = df_melted_days.groupby(['location']).rolling(7)['new_cases'].mean().reset_index(drop=True)
df_melted_days['new_cases_pm_ma'] = df_melted_days.groupby(['location']).rolling(7)['total_cases_per_million'].mean().reset_index(drop=True)

In [ ]:
df_melted_days

,location,days,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,new_cases_ma,new_cases_pm_ma
0,Afghanistan,0,75.0,33.0,1.0,0.0,1.927,0.848,0.026,0.000,NaN,NaN
1,Afghanistan,1,75.0,0.0,1.0,0.0,1.927,0.000,0.026,0.000,NaN,NaN
2,Afghanistan,2,91.0,16.0,2.0,1.0,2.338,0.411,0.051,0.026,NaN,NaN
3,Afghanistan,3,106.0,15.0,3.0,1.0,2.723,0.385,0.077,0.026,NaN,NaN
4,Afghanistan,4,114.0,8.0,4.0,1.0,2.928,0.206,0.103,0.026,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12450,Vietnam,100,332.0,0.0,0.0,0.0,3.411,0.000,0.000,0.000,0.571429,3.394857
12451,Vietnam,101,332.0,0.0,0.0,0.0,3.411,0.000,0.000,0.000,0.571429,3.400714
12452,Vietnam,102,333.0,1.0,0.0,0.0,3.421,0.010,0.000,0.000,0.571429,3.406571
12453,Vietnam,103,334.0,1.0,0.0,0.0,3.431,0.010,0.000,0.000,0.714286,3.413857
